# **Fine-tuning de FLAN-T5 pour la traduction Anglais-Francais**

Nous allons fine-tuner le modèle FLAN-T5 pour la traduction de l'anglais vers le français en utilisant avec Quantization + LoRA. 

Nous utilisons les bibliothèques peft et transformers pour l'entraînement.

- [Détails du modèle FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5)
- [Carte du modèle](https://huggingface.co/google/flan-t5-base)



In [ ]:
%%capture                     

!pip install transformers     ### pré-installé par défaut sur un notebook Kaggle
!pip install bitsandbytes 
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install peft
!pip install gradio

## Bibliothèques & Installations 🗂

In [ ]:
import os
import torch
import evaluate
import numpy as np
from huggingface_hub import notebook_login
from datasets import get_dataset_config_names, load_dataset
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, 
                          BitsAndBytesConfig,
                          Seq2SeqTrainer, 
                          Seq2SeqTrainingArguments, 
                          Trainer)

from peft import (PeftModel, 
                  PeftConfig,
                  LoraConfig, 
                  prepare_model_for_kbit_training, 
                  get_peft_model,
                  TaskType)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device --> {device}")

In [ ]:
### se connecter à son compte HF avec un token
notebook_login()

## Préparation des données 🛢

Nous allons utiliser le dataset [OPUS-100](https://huggingface.co/datasets/Helsinki-NLP/opus-100) pour l'entraînement.

OPUS-100 est un corpus multilingue centré sur l'anglais couvrant 100 langues.

Nous utiliserons juste les données "en-fr" pour la traduction de l'anglais au francais.

In [ ]:
configs = get_dataset_config_names("Helsinki-NLP/opus-100")
print(configs)

In [ ]:
dataset = load_dataset("Helsinki-NLP/opus-100", "en-fr")
dataset

In [ ]:
dataset['test']["translation"][:3]

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

### voir les détails du tokenizer
#tokenizer

In [ ]:
train_dataset = dataset['train'].shuffle(seed=42).select(range(5000))  ### échantillonner le jeu de données
eval_dataset = dataset['validation']

def preprocess_func(data):
    inputs  = [ex['en'] for ex in data['translation']]
    targets = [ex['fr'] for ex in data['translation']]
    
    ### tokenization de chaque ligne
    inputs = tokenizer(inputs,  truncation=True)
    labels = tokenizer(targets, truncation=True)
    
    inputs["labels"] = labels["input_ids"]
    
    return inputs


### tokenization du dataset entier
train_dataset = train_dataset.map(preprocess_func, batched=True)
eval_dataset  =  eval_dataset.map(preprocess_func, batched=True)

In [ ]:
train_dataset[0]

## Préparation du modèle 🦾

Tout d'abord, nous allons charger le modèle avec une quantification en 8 bits. Ensuite, nous utiliserons LoRA. 


Un rappel sur les paramètres de `LoraConfig` :

* **r** : le rang des matrices de mise à jour, exprimé en **int**. Un rang inférieur donne des matrices de mise à jour plus petites avec moins de paramètres entraînables.
* **target_modules** : **modules (par exemple, les blocs d'attention)** auxquels appliquer les matrices de mise à jour LoRA.
* **alpha** : facteur/taux de mise à l'échelle de LoRA.
* **bias** : spécifie si les paramètres de biais doivent être entraînés (peut être 'none', 'all' ou 'lora_only')
* **lora_dropout** : La probabilité de dropout des couches LoRA. Cela favorise la régularisation et évite l'overfitting en désactivant certains neurones.

In [ ]:
"""compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(load_in_4bit=True, 
                                bnb_4bit_quant_type="nf4", 
                                bnb_4bit_compute_dtype=compute_dtype,
                                bnb_4bit_use_double_quant=True,)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, 
                                              #quantization_config=bnb_config, 
                                              device_map=device,
                                              torch_dtype=compute_dtype,)"""

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, 
                                              quantization_config=bnb_config, 
                                              device_map=device,)

In [ ]:
### pour voir l'architecture technique du modèle
#model

In [ ]:
texts = tokenizer(["i love you so much, how have you been?",
                    "Steven, why don't you read it?",
                     "I'm so sorry that I didn't come home when you asked me to."
                    ], 
                    return_tensors="pt", padding=True).to(device)

texts

In [ ]:
output  = model.generate(**texts)
output

In [ ]:
tokenizer.batch_decode(output, skip_special_tokens=True)

In [ ]:
model = prepare_model_for_kbit_training(model)                       ### geler les paramètres d'origine et preparer le modele

peft_config = LoraConfig(r=32,                                       ### rang des matrices de mise à jour (low-rank matrices)
                        task_type=TaskType.SEQ_2_SEQ_LM,             ### type de tache (sequence-to-sequence language modeling dans ce cas)
                        lora_alpha=64,                               ### facteur de mise à l'échelle
                        lora_dropout=0.01,                           ### probabilité de dropout
                        target_modules=["k","q","v","o"])

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

## Fine-tuning du modèle 🦾


- [Importance du paramètre `auto_find_batch_size`](https://stackoverflow.com/questions/76359515/hugging-face-transformers-trainer-per-device-train-batch-size-vs-auto-find-batc)
-

In [ ]:
data_collator = DataCollatorForSeq2Seq( tokenizer=tokenizer, 
                                        model=peft_model, 
                                        label_pad_token_id= -100,        ### default value to ignore tokenizer pad token in the loss
                                        pad_to_multiple_of=8
                                      )

In [ ]:
training_args = Seq2SeqTrainingArguments(output_dir="./output",
                                        num_train_epochs=2,
                                        max_steps=-1,
                                        learning_rate=1e-3,              ### higher learning rate
                                        per_device_train_batch_size=8,   ### taille initiale du batch (pour chaque GPU, 1 dans notre cas)
                                        per_device_eval_batch_size=8,
                                        auto_find_batch_size=True,       ### optionnel (automates lowering of batch size to resolve out-of-memory errors)
                                        gradient_accumulation_steps=4, 
                                        gradient_checkpointing=True,     ### utilise le point de contrôle de gradient pour économiser de la mémoire
                                        optim="paged_adamw_32bit",
                                        logging_strategy="steps",
                                         
                                        weight_decay=0.001,
                                        #fp16=True,
                                        #bf16=False,
                                        max_grad_norm=0.3,                ### norme maximale du gradient basée sur le papier QLoRA
                                        warmup_ratio=0.03,                ### ratio de préchauffage basé sur le papier QLoRA
                                        lr_scheduler_type="linear",       ### utilise le programmeur de taux d'apprentissage linéaire
                                        report_to="tensorboard",          ### rapporte les métriques à tensorboard
                                        logging_steps=10,
                                        save_strategy="no",)


trainer = Seq2SeqTrainer(model=peft_model,
                        args=training_args,
                        data_collator=data_collator,
                        train_dataset=train_dataset,
                        eval_dataset=eval_dataset,
                        compute_metrics=compute_metrics,)

peft_model.config.use_cache=False

In [ ]:
trainer.train()

In [ ]:
#trainer.evaluate() # out of GPU memory

In [ ]:
!pip install -qU tensorboard

%load_ext tensorboard
%tensorboard --logdir logs/runs

In [ ]:
### enregistrer le modele localement
peft_model.save_pretrained("translation-en2fr")
tokenizer.save_pretrained("translation-en2fr")

In [ ]:
merged_model = PeftModel.from_pretrained(model, "translation-en2fr")
merged_model = merged_model.merge_and_unload()

In [ ]:
### pusher le modele sur Hugging Face
tokenizer.push_to_hub("anyantudre/flan-t5-ft-en2fr")
merged_model.push_to_hub("anyantudre/flan-t5-ft-en2fr")

# Inference

In [ ]:
texts = tokenizer(["i love you so much, how have you been?",
                    "Steven, why don't you read it?",
                     "I'm so sorry that I didn't come home when you asked me to."
                    ], 
                    return_tensors="pt",
                     padding="longest").to(device)
output  = merged_model.generate(**texts)

tokenizer.batch_decode(output, skip_special_tokens=True)

# Interface Gradio

In [ ]:
import gradio as gr
from transformers import pipeline

model_checkpoint = "anyantudre/flan-t5-ft-en2fr"

my_input  = gr.Textbox(label="Input")
my_output = gr.Textbox(label="Translation")

def func(source_text):
    translation = translator(source_text)
    return translation

demo=gr.Interface(
   fn=func,
   inputs=my_input,
   outputs=my_output
)
demo.launch()